In [1]:
### DON'T DO "RESTART AND RUN ALL CELLS" ON THIS NOTEBOOK ###
### THERE'S A CELL THAT TAKES VERY LONG TO RUN AND IT ONLY NEEDED TO BE RUN ONCE ###

In [2]:
import pandas as pd
import numpy as np
import requests
import string
import collections
import random
import time
import beepy
from tqdm import tqdm
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.word2vec import Word2Vec
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess
from gensim.models.phrases import Phrases, ENGLISH_CONNECTOR_WORDS
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textaugment import Wordnet

In [3]:
# get dataframes from CSV files

df_occupations = pd.read_csv('../data/ESCO/occupations_en.csv')
df_occupations_aug = pd.read_csv('../data/ESCO/occupations_augmented.csv')
df_occ_n_skills = pd.read_csv('../data/ESCO/occupations_augmented_with_skills.csv')

/var/folders/84/1j64f6yd3yd5g68_pj91ch780000gn/T/ipykernel_10448/3036819475.py:5: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_occ_n_skills = pd.read_csv('../data/ESCO/occupations_augmented_with_skills.csv')


In [5]:
# create description_input and skills_input, which are strings on which the model will be fit

for row, index in tqdm(df_occ_n_skills.iterrows()):
    underscored_job_title = index['job_title'].replace(" ", "_")
    this_rows_description_input = underscored_job_title + ' ' + index['description']
    this_rows_skills_input = underscored_job_title + ' ' + index['skills']
    df_occ_n_skills.iloc[row,-2] = this_rows_description_input
    df_occ_n_skills.iloc[row,-1] = this_rows_skills_input

35824it [00:06, 5433.46it/s]


In [6]:
# show dataframe head

df_occ_n_skills.head()

,job_title,description,skills,description_input,skills_input
0,technical director,Technical directors realise the artistic visio...,"adapt to artists' creative demands, organise r...",technical_director Technical directors realise...,technical_director adapt to artists' creative ...
1,metal drawing machine operator,Metal drawing machine operators set up and ope...,"cold drawing processes, monitor moving workpie...",metal_drawing_machine_operator Metal drawing m...,metal_drawing_machine_operator cold drawing pr...
2,precision device inspector,Precision device inspectors make sure precisio...,"precision measuring instruments, monitor machi...",precision_device_inspector Precision device in...,precision_device_inspector precision measuring...
3,air traffic safety technician,Air traffic safety technicians provide technic...,"air transport law, aircraft flight control sys...",air_traffic_safety_technician Air traffic safe...,air_traffic_safety_technician air transport la...
4,hospitality revenue manager,Hospitality revenue managers maximise revenue ...,"develop revenue generation strategies, ensure ...",hospitality_revenue_manager Hospitality revenu...,hospitality_revenue_manager develop revenue ge...


In [7]:
# get entire dataframe as dataset according to Qiewi's suggestion
# concatenate the job_title:skills list to the end of df_occ_n_skills

X_all = pd.concat([df_occ_n_skills['description_input'], df_occ_n_skills['skills_input']]).reset_index(drop=True)

In [9]:
# Preprocessing function for job descriptions

def preprocess_input(sentence):
    
    # remove punctuation
    for punctuation in string.punctuation:        
        sentence = sentence.replace(punctuation, '')
    
    # set lowercase
    sentence = sentence.lower()
    
    # remove numbers
    sentence = ''.join(char for char in sentence if not char.isdigit())
    
    # remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(sentence)
    stopword_free_tokens = [token for token in tokens if token not in stop_words]
    sentence = ' '.join(stopword_free_tokens)

    # lemmatize
    sentence = WordNetLemmatizer().lemmatize(sentence, pos='n')
    sentence = WordNetLemmatizer().lemmatize(sentence, pos='v')
    
    # split into tokens again after Lemmatizing --- this was replaced by Phraser 
    # sentence = word_tokenize(sentence)
    
    return sentence

In [10]:
# applying preprocessing as in Gensim tutorial, also applying Phraser

def read_corpus(corpus):
   
    # instantiate Phraser outside of the loop
    sentence_stream = [entry.split(" ") for entry in corpus]
    bigrams = Phrases(
        sentence_stream,
        min_count=5,
        threshold=5,
        connector_words=ENGLISH_CONNECTOR_WORDS
        )
    
    for i, line in enumerate(corpus):    
    
        # remove punctuation
        for punctuation in string.punctuation:        
            sentence = line.replace(punctuation, '')

        # remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = word_tokenize(sentence)
        stopword_free_tokens = [token for token in tokens if token not in stop_words]
        sentence = ' '.join(stopword_free_tokens)

        # lemmatize
        sentence = WordNetLemmatizer().lemmatize(sentence, pos='n')
        sentence = WordNetLemmatizer().lemmatize(sentence, pos='v')
        
        # get bigrams
        sent = sentence.split()

        # yield tagged final corpus
        yield TaggedDocument(bigrams[sent], [i])

all_corpus = list(read_corpus(X_all))

In [27]:
# initialize 'job2vec' model

job2vec_model = Doc2Vec(         # these are the parameters that got the best results so far:
    vector_size=80,              # 80
    min_count=1,                 # 1
    epochs=100,                  # 100
    window=5,                    # 5
    alpha=0.01,                  # 0.01
    dm_concat=1,                 # 1
    workers=8,                   # 8
    shrink_windows=True          # True
    )

# build vocabulary
job2vec_model.build_vocab(all_corpus)

job2vec_model.wv

In [13]:
### train model! ###

job2vec_model.train(
    corpus_iterable=all_corpus,
    total_examples=job2vec_model.corpus_count,
    epochs=job2vec_model.epochs
    )

In [23]:
# describe your job and get your prediction!

describe_your_job = \
"Army majors command large units of officers and soldiers, supervise their training, and oversee their wellfare. They also supervise their administration, and equipment management"


In [24]:
# preprocess your job description
new_description = preprocess_input(describe_your_job)

# detect bigrams with Gensim Phrases
sentence_stream = [entry.split(" ") for entry in X_all]
bigrams = Phrases(
    sentence_stream,
    min_count=5,
    threshold=5,
    connector_words=ENGLISH_CONNECTOR_WORDS
    )
sent = new_description.split()

# create inferred vector from your preprocessed job description
new_inferred_vector = job2vec_model.infer_vector(bigrams[sent])

# get similar job descriptions from model
similar_to_new = job2vec_model.dv.most_similar([new_inferred_vector])

In [25]:
print(f'Test Document: {new_description} \n')
print('LISTING 3 MOST SIMILAR JOB ROLES & DESCRIPTIONS \n')

for label, index in [('* MOST SIMILAR', 0), ('* 2ND MOST SIMILAR', 1), ('* 3RD MOST SIMILAR', 2)]:
    
    if similar_to_new[index][0] <= len(df_occ_n_skills):
        new_index = similar_to_new[index][0]
    else:
        new_index = similar_to_new[index][0] - len(df_occ_n_skills)

    print(label + ': ' + df_occ_n_skills.loc[new_index]['job_title'])
    print(df_occ_n_skills.loc[new_index]['description'])
    print(f'Similarity score: {round(similar_to_new[index][1]*100,1)} %')
    print(f'Index in dataframe: {new_index} \n')


Test Document: management consultancy specialized food beverage consulting services restaurants bars similar ensure permanent inventory maintenance controlling stock levels stock replenishment 

LISTING 3 MOST SIMILAR JOB ROLES & DESCRIPTIONS 

* MOST SIMILAR: dairy plant worker
dairy production manufacturing worker set up, operate and tend equipment to process milk, cheese, ice emollient and other dairy products.
Similarity score: 78.5 %
Index in dataframe: 21177 

* 2ND MOST SIMILAR: blancher
blanching operator remove outer coverings or pelt from almonds and junky in general. they cut leaf and impurity of raw fabric and control the menstruum of nuts, seeds, and/or leaves in the process. they use pressure and temperature to blanch the raw material if necessary.
Similarity score: 73.9 %
Index in dataframe: 18104 

* 3RD MOST SIMILAR: food science docent
nutrient skill lector are subject professors, teachers, or lecturers who instruct bookman who have obtained an upper secondary trainin

In [26]:
beepy.beep(sound="ping")